In [1]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import lightgbm as lgb
import warnings
import gc
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import metrics
warnings.simplefilter('ignore')

In [36]:
# dataの読み込み
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
submit_df = pd.read_csv('./submit_sample.csv',header=None)

In [37]:
# データの量の確認
train_df.shape,test_df.shape,submit_df.shape

((27100, 18), (18050, 17), (18050, 2))

In [38]:
# 訓練データ、テストデータがわかるようにダミーの目的変数を代入
test_df['y']=-999

In [39]:
# 訓練データ、テストデータを結合
all_df = pd.concat([train_df,test_df])
del train_df,test_df
gc.collect()

146

In [40]:
# カテゴリカラムの前処理
categorical_features = ['job', 'marital', 'education','default','housing','loan','contact','month','poutcome']
for col in categorical_features:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(all_df[col])
    lbl.transform(all_df[col])
    all_df[col]=lbl.transform(all_df[col])

In [41]:
#fe = ['age', 'marital', 'education', 'housing', 'loan', 'contact' ,'month', 'campaign', 'previous', 'poutcome']
#all_df['log_bal'] = np.log(all_df['balance'])
#all_df['bal_div_age'] = all_df['balance'] / all_df['age']
#all_df['bal_mul_age'] = all_df['balance'] * all_df['age']
#all_df['bal_div_age_p3'] = np.power(all_df['bal_div_age'], 3)
#sall_df['campaign_p2'] = np.power(all_df['campaign'], 2)
all_df['housing_and_loan'] = all_df['housing'] & all_df['loan']
all_df['month_q'] = all_df['month'] / 3



#all_df.drop(['job', 'balance', 'day', 'duration', 'pdays'], axis=1)


In [68]:
def link_mean_job():
    mean_job_df = all_df.groupby(['job'], as_index=False).mean()
    all_df['sub_job_mean'] = 0
    for i in range(len(all_df['job'].value_counts())):
        tmp = np.array(all_df[(all_df['job'] == i)]['balance']) - np.array(mean_job_df[mean_job_df['job'] == 0]['balance'])
        all_df[(all_df['job'] == i)]['sub_job_mean'] = tmp
        print(np.sum)
        print("sub_job___", np.sum(all_df['sub_job_mean']))

In [50]:
mean_job_df = all_df.groupby(['job'], as_index=False).mean()
print(mean_job_df[mean_job_df['job'] == 0]['balance'])

0    47253.40944
Name: balance, dtype: float64


In [65]:
len(all_df['job'].value_counts())

12

In [69]:
link_mean_job()

sub_job___ 0        0
1        0
2        0
3        0
4        0
        ..
18045    0
18046    0
18047    0
18048    0
18049    0
Name: sub_job_mean, Length: 45150, dtype: int64
sub_job___ 0        0
1        0
2        0
3        0
4        0
        ..
18045    0
18046    0
18047    0
18048    0
18049    0
Name: sub_job_mean, Length: 45150, dtype: int64
sub_job___ 0        0
1        0
2        0
3        0
4        0
        ..
18045    0
18046    0
18047    0
18048    0
18049    0
Name: sub_job_mean, Length: 45150, dtype: int64
sub_job___ 0        0
1        0
2        0
3        0
4        0
        ..
18045    0
18046    0
18047    0
18048    0
18049    0
Name: sub_job_mean, Length: 45150, dtype: int64
sub_job___ 0        0
1        0
2        0
3        0
4        0
        ..
18045    0
18046    0
18047    0
18048    0
18049    0
Name: sub_job_mean, Length: 45150, dtype: int64
sub_job___ 0        0
1        0
2        0
3        0
4        0
        ..
18045    0
18046    0
1

In [55]:
print(all_df.head())

   id  age  job  marital  education  default  balance  housing  loan  contact  \
0   0   31    7        1          1        0    12294        1     0        0   
1   1   29    2        2          2        0    43027        0     0        0   
2   2   35    4        1          2        0    12252        1     0        0   
3   3   31    9        1          1        0    99121        1     1        2   
4   4   48   10        1          0        0    42005        1     0        1   

   day  month  duration  campaign  pdays  previous  poutcome  y  \
0   21      8       101         3    498         0         1  0   
1   22      1       158         2    702         0         3  1   
2   11      8       351         1    826         0         0  0   
3   16      7       658         2    120         0         0  0   
4    3      0       177         1    273         0         3  0   

   housing_and_loan   month_q  
0                 0  2.666667  
1                 0  0.333333  
2             

In [ ]:
# 訓練データ、テストデータの分割
train_df = all_df[all_df['y']!=-999]
test_df = all_df[all_df['y']==-999]

In [ ]:
y_train = train_df['y']
X_train = train_df.drop(['y','id'], axis=1)
X_test = test_df.drop(['y','id'], axis=1)

In [ ]:
from sklearn.model_selection import KFold
from sklearn import metrics


y_preds = []
models = []
oof_train = np.zeros((len(X_train),))
cv = KFold(n_splits=5, shuffle=True, random_state=0)


params = {
    'objective': 'binary',
    'max_bin': 300,
    'learning_rate': 0.05,
    'num_leaves': 40
}

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr, categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train, categorical_feature=categorical_features)

    model = lgb.train(
        params, lgb_train,
        valid_sets=[lgb_train, lgb_eval],
        verbose_eval=10,
        num_boost_round=1000,
        early_stopping_rounds=10
    )

    oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    y_preds.append(y_pred)
    models.append(model)
    
    # eval
    #acc = accuracy_score(y_val, y_pred)
    fpr, tpr, thresholds = metrics.roc_curve(y_val, oof_train[valid_index])
    auc = metrics.auc(fpr, tpr)
    print('auc ', auc)

In [ ]:
pd.DataFrame(oof_train).to_csv('oof_train_kfold.csv', index=False)

scores = [
    m.best_score['valid_1']['binary_logloss'] for m in models
]
score = sum(scores) / len(scores)
print('===CV scores===')
print(scores)
print(score)

In [ ]:
from sklearn.metrics import accuracy_score


y_pred_oof = (oof_train > 0.5).astype(int)
accuracy_score(y_train, y_pred_oof)

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_train, oof_train)
metrics.auc(fpr, tpr)

# baseline is 0.8512576188085433
# 0.8517739935623644

In [ ]:
from sklearn.metrics import confusion_matrix


cm = confusion_matrix(y_train, y_pred_oof)

In [ ]:
cm

## Reference
- CV
    - https://www.kaggle.com/sishihara/upura-kaggle-tutorial-06-validation
- Metric
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.auc.html#sklearn.metrics.auc